# Lab 4-1: Multivariable Linear Regression

Author: Seungjae Lee (이승재)

<div class="alert alert-warning">
    nn.Module 과 nn.Linear 대신 pure PyTorch로 해볼까?
    ... 해야 matrix 부분을 소개할 수 있을것 같다.
</div>

## Hypothesis and cost function

$$ H(x_1, x_2, x_3) = x_1w_1 + x_2w_2 + x_3w_3 + b $$

$$ cost(W, b) = \frac{1}{m} \sum^m_{i=1} \left( H(x^{(i)}) - y^{(i)} \right)^2 $$

 - $H(x)$: 주어진 $x$ 값에 대해 예측을 어떻게 할 것인가
 - $cost(W, b)$: $H(x)$ 가 $y$ 를 얼마나 잘 예측했는가

## PyTorch

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# For reproducibility
torch.manual_seed(1)

우선 피팅할 가짜 데이터를 만들자.

In [3]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

기본적으로 PyTorch는 가장 먼저 NCHW 형태이다.

In [4]:
x_train.shape

torch.Size([5, 3])

In [5]:
y_train.shape

torch.Size([5, 1])

이제 linear regression 모델을 만들면 되는데, 기본적으로 PyTorch의 모든 모델은 제공되는 `nn.Module`을 inherit 해서 만들게 됩니다.

In [6]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        return self.linear(x)

모델의 `__init__`에서는 사용할 레이어들을 정의하게 됩니다. 여기서 우리는 linear regression 모델을 만들기 때문에, `nn.Linear` 를 이용할 것입니다. 그리고 `forward`에서는 이 모델이 어떻게 입력값에서 출력값을 계산하는지 알려줍니다.

이제 모델을 생성해서 예측값 $H(x)$를 구해보자

In [7]:
model = LinearRegressionModel()

In [8]:
prediction = model(x_train)

In [9]:
print(prediction)

tensor([[-6.7933],
        [-4.8968],
        [-6.5155],
        [-7.3361],
        [-2.6660]], grad_fn=<ThAddmmBackward>)


이제 mean squared error (MSE) 로 cost를 구해보자. MSE 역시 PyTorch에서 기본적으로 제공한다.

In [10]:
print(prediction)
print(y_train)

tensor([[-6.7933],
        [-4.8968],
        [-6.5155],
        [-7.3361],
        [-2.6660]], grad_fn=<ThAddmmBackward>)
tensor([[152.],
        [185.],
        [180.],
        [196.],
        [142.]])


In [11]:
cost = F.mse_loss(prediction, y_train)

In [12]:
print(cost)

tensor(31667.5977, grad_fn=<MseLossBackward>)


마지막 주어진 cost를 이용해 $H(x)$ 의 $W, b$ 를 바꾸어서 cost를 줄여봅니다. 이때 PyTorch의 `torch.optim` 에 있는 `optimizer` 들 중 하나를 사용할 수 있습니다.

In [13]:
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [14]:
optimizer.zero_grad()
cost.backward()
optimizer.step()

이제 Linear Regression 코드를 이해했으니, 실제로 코드를 돌려 피팅시켜보겠습니다.

In [22]:
# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])
# 모델 초기화
model = LinearRegressionModel()
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 1000
for epoch in range(nb_epochs):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 100 == 99:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch+1, nb_epochs, cost.item()
        ))

Epoch  100/1000 Cost: 0.954945
Epoch  200/1000 Cost: 0.914483
Epoch  300/1000 Cost: 0.876153
Epoch  400/1000 Cost: 0.839847
Epoch  500/1000 Cost: 0.805431
Epoch  600/1000 Cost: 0.772845
Epoch  700/1000 Cost: 0.741979
Epoch  800/1000 Cost: 0.712734
Epoch  900/1000 Cost: 0.685022
Epoch 1000/1000 Cost: 0.658776


점점 $H(x)$ 의 $W$ 와 $b$ 를 조정해서 cost가 줄어드는 것을 볼 수 있습니다.

<div class="alert alert-warning">
    W 와 b 값을 출력해볼까?
</div>